# Esqueleto para o Algorimo de Recomendação

Ações:
treinar, salvar modelo, carregar modelo e predição

```
# Isto está formatado como código
```



In [ ]:
import uuid

def train_model(csv_filename):
  print('read csv and train the model, grid search is optional')
  model = 'final model'
  return model

def save_model(model):
  # nome auto gerado
  # model_filename = 'c3256d66-e1cb-45fa-8ccd-c39355466c03.model'
  model_filename = str(uuid.uuid4()) + '.model'
  print('something like model.save(model_filename)')
  return model_filename

def load_model(model_filename):
  print('something like model = something.load(model_filename)')
  model = 'loaded model'
  return model

def predict(model, product_id):
  print('something model.predict([product_id])')
  result = [12, 48, 98, 88, 99]
  return result

# Teste e uso no Anvil

In [ ]:
# part 1 - training
csv_filename = 'casasbahia.csv'
print('training model...')
model = train_model(csv_filename)
print('saving model...')
model_filename = save_model(model)
print(f'model saved to {model_filename}')

# part 2 - predicting
print('loading saved model from disk...')
model = load_model(model_filename)
product_id = 25
print(f'predicting product {product_id}..')
result = predict(model, product_id)
print(f'prediction result={result}')

training model...
read csv and train the model, grid search is optional
saving model...
something like model.save(model_filename)
model saved to dda1620d-9ead-4e1d-8319-bdea7b0e7cea.model
loading saved model from disk...
something like model = something.load(model_filename)
predicting product 25..
something model.predict([product_id])
prediction result=[12, 48, 98, 88, 99]


## Conexão com o Anvil


In [ ]:
#!pip install anvil-uplink
import anvil.server
anvil.server.connect("BFSI2NTMXOF7JIPPVF5FMCBY-55DIUIXG7VVL3GVJ")

In [ ]:
############### thread python para o serviço do Anvil rodar em background ########## opcional

from threading import Thread
import time


def thread_anvil():
  print('Inicio da thread...')    
  anvil.server.wait_forever() 
  print('Thread do serviço Anvil acionada...')

print('Iniciando a thread do serviço Anvil...')

# criar e executa a thread
#t = Thread(target=thread_anvil, args=())
#t.start()
    
print('Thread do serviço Anvil acionada...')


Iniciando a thread do serviço Anvil...
Thread do serviço Anvil acionada...


## Recebe o Arquivo

In [ ]:
import anvil.media

@anvil.server.callable
def receive_file(file):
  with anvil.media.TempFile(file) as filename:
    img = load_image(filename)

    ....

# VERSÃO FINAL COM SURPRISE

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 17.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619419 sha256=d5261a1e9c2807cde6bd225cb6d5469973967f8faa797b5fbb573da79ac02f3c
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import uuid
import sys
import os
import pickle
import time
from collections import defaultdict
from surprise.prediction_algorithms import knns
from surprise import SVD
from surprise import Dataset
from threading import Thread


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        if n:
          top_n[uid] = user_ratings[:n]
        else:
          top_n[uid] = user_ratings

    return top_n


def train_model(csv_filename, n=None):
  start = time.time()
  # TODO: alterar para ler .csv
  # print('loading from dataset')
  # df = pd.read_csv(csv_filename)
  # data = Dataset.load_from_df(df[['UserId', 'ProductId', 'Rating']], Reader(rating_scale=(1,5)))
  data = Dataset.load_builtin('ml-100k')
  trainset = data.build_full_trainset()
  algo = SVD(random_state=1)
  algo.fit(trainset)

  # Than predict ratings for all pairs (u, i) that are NOT in the training set.
  testset = trainset.build_anti_testset()
  predictions = algo.test(testset)

  model = get_top_n(predictions, n=n)
  elapsed_seconds = time.time() - start
  print(f'model trained in {elapsed_seconds:.1f} seconds')
  save_model(model)
  return model

@anvil.server.callable
def thread_train_model(csv_filename, n=None):
# criar e executa a thread
  t = Thread(target=train_model, args=(csv_filename, n))
  t.start()  
  print('Thread do serviço de treinamento acionada...')
  return t.getName

def save_model(model):
  tamanho_modelo = sys.getsizeof(model)
  print(f'tamanho do modelo: {tamanho_modelo} bytes')
  file_name = str(uuid.uuid4()) + '.model'
  pickle.dump(model, open(file_name, 'wb'))
  tamanho_arquivo = os.stat(file_name).st_size
  print(f'tamanho do arquivo: {tamanho_arquivo} bytes')
  return file_name

def load_model(file_name):
  print('Carregando o modelo')
  file_name = file_name + '.model'
  loaded_model = pickle.load(open(file_name, 'rb'))
  print ("Modelo carregado")
  return loaded_model


@anvil.server.callable
def predict_user(model_filename, user_id, n=5):
  print(model_filename)
  model = load_model(model_filename)
  print('tamanho modelo: ', len(model))
  print(f'predicting user {user_id}..')
  result = []
  # Print the recommended items for each user
  for uid, user_ratings in model.items():
    print('uid: ', uid)
    if str(uid) == str(user_id):
      result = [iid for (iid, _) in user_ratings][:n]
  print ('Predict_user2:', result)
  return result  

@anvil.server.callable
def predict_item(model_filename, item_id, n=5):
  print('Predicting for item: ', item_id)
  model = load_model(model_filename)
  result = []
  # Print the recommended items for each user
  print('step 1')
  top_n_iid = defaultdict(list)
  for uid, user_ratings in model.items():
    for (iid, rating) in user_ratings:
      if str(iid) == str(item_id):
        top_n_iid[iid].append((uid, rating))   
  print('top_n_iid:')
  print(top_n_iid)           
  print('step 2')
  # Then sort the predictions for each user and retrieve the k highest ones.
  for iid, user_ratings in top_n_iid.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
  print('step 3')
  result = [iid for (iid, _) in user_ratings][:n]
  print('result: ')
  print(result)
  return result  


@anvil.server.callable
def predict_user_item(model_filename, user_id, item_id):
  model = load_model(model_filename)
  result = []
  # Print the recommended items for each user
  for uid, user_ratings in model.items():
    if str(uid) == str(user_id):
      for (iid, rating) in user_ratings:
        if str(iid) == str(item_id):
          result.append(rating)
  return result  



In [ ]:
# part 2 - predicting
# print(model_filename)
model = load_model(model_filename)
user_id = 196
print(f'predicting user {user_id}..')
result_user = predict_user(model_filename, user_id, n=5)
print(f'prediction result={result_user}')


In [ ]:
################### aqui roda o serviço e faz todos os prints das APIs acionadas ####### essa é a melhor opção para rodas
anvil.server.wait_forever()


Predicting for item:  98
Carregando o modelo
Modelo carregado
step 1
top_n_iid:
defaultdict(<class 'list'>, {'98': [('196', 4.6788753121780635), ('22', 4.695825459687281), ('244', 4.55397707888762), ('166', 4.387311362228051), ('286', 4.8236920474947), ('224', 4.06359872134985), ('122', 4.587093950293811), ('119', 4.559849491116247), ('167', 4.1380079404187615), ('38', 4.37488916014183), ('63', 3.9032836204227555), ('160', 4.823509371300489), ('50', 4.7136145526077735), ('157', 5), ('181', 2.7278056871620526), ('284', 4.494913849097564), ('242', 4.684088400431999), ('251', 4.784624344229448), ('260', 4.892963732541499), ('87', 4.6010178305445555), ('57', 4.754425691531723), ('223', 4.385917347833458), ('189', 4.337727874297825), ('243', 4.209806829163393), ('241', 4.322181791438202), ('127', 4.895241084218646), ('8', 4.376824220592425), ('162', 4.173571757201392), ('279', 3.359962383678984), ('32', 4.181634776473936), ('265', 4.585850589985849), ('168', 4.596132736455048), ('110', 4.32

## Training

In [ ]:
# part 1 - training
csv_filename = 'ratings_Beauty.csv'
print('training model...')
model = train_model(csv_filename)
print('saving model...')
model_filename = save_model(model)
print(f'model saved to {model_filename}')

training model...
Dataset ml-100k could not be found. Do you want to download it? [Y/n] 
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
model trained in 26.2 seconds
saving model...
tamanho do modelo: 36984 bytes
tamanho do arquivo: 62238699 bytes
model saved to 21649d8e-b8da-4d06-8fcb-43a4ce369fe9.model


## Prediction

In [ ]:
# part 2 - predicting
print('loading saved model from disk...')
print(model_filename)
model = load_model(model_filename)
#print('modelo: ', model)
user_id = 196
print(f'predicting user {user_id}..')
result_user = predict_user(model_filename, user_id, n=5)
print(f'prediction result={result_user}')

if len(result_user) > 0:
  item_id = result_user[0]
  print(f'predicting item {item_id}..')
  result_item = predict_item(model, item_id, n=5)
  print(f'prediction result={result_item}')
  if len(result_item) > 0:
    print(f'predicting item {item_id} for user {user_id}..')
    result_user_item = predict_user_item(model, user_id, item_id)
    print(f'prediction result={result_user_item}')  

loading saved model from disk...
21649d8e-b8da-4d06-8fcb-43a4ce369fe9.model
Carregando o modelo
Modelo carregado
predicting user 196..
loading saved model from disk...
21649d8e-b8da-4d06-8fcb-43a4ce369fe9.model
Carregando o modelo
Modelo carregado
predicting user 196..
tamanho modelo:  943
Predict_user2: ['98', '64', '408', '318', '483']
prediction result=['98', '64', '408', '318', '483']
predicting item 98..
prediction result=['157', '173', '4', '252', '261']
predicting item 98 for user 196..
prediction result=[4.6788753121780635]
